In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

In [14]:
# URL to scrape

# url = "https://events.cmu.edu/day/date/20241006"

f = open("NEWFILE.txt", "w+")

# Request the webpage content
def get_data_from_url(url, month, date, year, file = f):
    
    # response = requests.get(url, verify = False)
    # soup = BeautifulSoup(response.text, 'html.parser')
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='lw_cal_event_list']")))
    # Get the page source after interactions
    page_source = driver.page_source
    # Parse the page source with Beautiful Soup
    soup = BeautifulSoup(page_source, "html.parser")

    events = soup.find_all('div', class_='lw_cal_event')
  
    # print(events)

    for event in events:
        event_title = event.find('div', class_='lw_events_title').find('a').get_text(strip=True) if event.find('div', class_='lw_events_title') else "N/A"

        # Event link
        event_link = event.find('div', class_='lw_events_title').find('a')['href'] if event.find('div', class_='lw_events_title') else "N/A"

        # Event time
        start_time = event.find('span', class_='lw_start_time').get_text(strip=True) if event.find('span', class_='lw_start_time') else "N/A"
        end_time = event.find('span', class_='lw_end_time').get_text(strip=True) if event.find('span', class_='lw_end_time') else "N/A"
        event_time = f"{start_time} - {end_time}"
        event_date = str(month) + "/" + str(date) + "/" + str(year)

        # Event venue
        event_venue = event.find('div', class_='lw_events_location').get_text(strip=True) if event.find('div', class_='lw_events_location') else "N/A"

        # Event description
        event_description = event.find('div', class_='lw_events_summary').get_text(strip=True) if event.find('div', class_='lw_events_summary') else "N/A"

        # Image link (optional if present)
        # image = event.find('img')['src'] if event.find('img') else 'No Image'

        # Print or store the event details
        # print({
        #     'Title': event_title,
        #     'Link': event_link,
        #     'Time': event_time,
        #     'Venue': event_venue,
        #     'Description': event_description
        #     # 'Image': image
        # })
        f.write(f"Event: {event_title}\nLink: {event_link}\nTime: {event_date}, {event_time}\nLocation: {event_venue}\nDescription: {event_description}\n\n")
        print(f"Event: {event_title}\nLink: {event_link}\nTime: {event_date}, {event_time}\nLocation: {event_venue}\nDescription: {event_description}\n\n")

# get_data_from_url("https://events.cmu.edu/", 10, 6, 2024)
for i in range(10,13):
    for j in range(1,10):
        date = str(i) + "0" + str(j)
        url = "https://events.cmu.edu/day/date/2024" + date
        get_data_from_url(url, i, j, 2024)
    for j in range(20,32):
        date = str(i) + str(j)
        url = "https://events.cmu.edu/day/date/2024" + date
        get_data_from_url(url, i, j, 2024)

for i in range(1,10):
    for j in range(1,10):
        date = "0" + str(i) + "0" + str(j)
        url = "https://events.cmu.edu/day/date/2025" + date
        get_data_from_url(url, i, j, 2025)
    for j in range(20,32):
        date = "0" + str(i) + str(j)
        url = "https://events.cmu.edu/day/date/2025" + date
        get_data_from_url(url, i, j, 2025)
        
# driver.quit()

Event: 17th International Exhibition of Botanical Art & Illustration
Link: /event/8412-17th-international-exhibition-of-botanical-art-amp
Time: 10/1/2024, 9 a.m. - 5 p.m.
Location: Hunt Institute for Botanical Documentation, 5th Fl...
Description: This Hunt Institute for Botanical Documentation exhibition includes 43 artworks by 43 artists representing 19 countries.


Event: Swartz Center for Entrepreneurship CONNECTS Seminar Series
Link: /event/10160-swartz-center-for-entrepreneurship-connects-seminar-se
Time: 10/1/2024, 12:30 - 1:30 p.m.
Location: Swartz Center for Entrepreneurship, 3rd Level, Tep...
Description: The CONNECTS seminar series enhances the experiential education of any individual interested in innovation and entrepreneurship.


Event: Immigration at the Ballot Box: Key Issue for 2024
Link: /event/10979-immigration-at-the-ballot-box-key-issue-for-2024
Time: 10/1/2024, 12:30 - 1:30 p.m.
Location: 
Description: This virtual panel will delve into why people migrate, the cri

In [52]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Base URL of the website
base_url = "https://www.sportspittsburgh.com/power-of-sports-economy/professional-sports-teams/"

f = open("sports_teams_3.txt", "w+")

# Send a request to the website
response = requests.get(base_url, verify=False)
print(response)

# Parse the content
soup = BeautifulSoup(response.text, "html.parser")

# Find all anchor tags
anchors = soup.find_all('a')

# Extract and format all subpage links
subpages = set()
for anchor in anchors:
    href = anchor.get('href')
    if href:
        full_url = urljoin(base_url, href)
        subpages.add(full_url)

def scrape_subpage_content(url):
    try:
        subpage_response = requests.get(url, verify=False)
        subpage_soup = BeautifulSoup(subpage_response.text, "html.parser")

        # Scrape relevant content (adjust based on site structure)
        content = subpage_soup.find_all('p')  # Example: scraping paragraphs
        page_content = "\n".join([p.get_text() for p in content])
        return page_content
    except Exception as e:
        return f"Error scraping {url}: {e}"

# Scrape each subpage
for subpage in subpages:
    print(f"Scraping {subpage}...")
    # if subpage == "https://mlb.tickets.com/?agency=PIRM_MYTIXX&orgid=39910#/auth/login":
    #     continue
    try:
        content = scrape_subpage_content(subpage)
        f.write(content)
    except:
        print("could not scrape " + subpage)
        continue

# content = scrape_subpage_content("https://www.heinzhistorycenter.org/events/")
# print(content)
# print(subpages)

<Response [200]>
Scraping https://www.sportspittsburgh.com/sports-planner/sports-event-support/...
Scraping https://www.sportspittsburgh.com/power-of-sports-economy/community-involvement/...
Scraping https://www.sportspittsburgh.com/about-sportspittsburgh/sportspittsburgh-team/...
Scraping https://destinationsinternational.org/...
Scraping https://www.sportspittsburgh.com/attendee-experiences/legendary-sports-city/...
Scraping http://www.trustyou.com/...
Scraping https://www.sportspittsburgh.com/newsletter-sign-up/...
Scraping https://www.sportspittsburgh.com/attendee-experiences/events-calendar/...
Scraping https://www.sportspittsburgh.com/...
Scraping https://www.visitpittsburgh.com/privacy/...
Scraping https://www.sportspittsburgh.com/power-of-sports-economy/professional-sports-teams/#main...
Scraping https://www.sportspittsburgh.com/attendee-experiences/ncaa-pittsburgh/...
Scraping https://www.sportspittsburgh.com/about-sportspittsburgh/...
Scraping https://www.visittheusa.com/...


In [56]:
# base_url = "https://www.nhl.com/penguins/"

f = open("random.txt", "w+")

# Send a request to the website
# response = requests.get(base_url, verify=False)
# print(response)

def scrape_subpage_content(url):
    try:
        # driver = webdriver.Chrome()
        # driver.get(url)
        # last_height = driver.execute_script("return document.body.scrollHeight")
        # while True:
        #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #     time.sleep(2)  # Wait for the page to load
        #     new_height = driver.execute_script("return document.body.scrollHeight")
        #     if new_height == last_height:
        #         break
        #     last_height = new_height
        # # Get the page source after interactions
        # page_source = driver.page_source
        # # Parse the page source with Beautiful Soup
        # soup = BeautifulSoup(page_source, "html.parser")
        subpage_response = requests.get(url, verify=False)
        soup = BeautifulSoup(subpage_response.text, "html.parser")

        # Scrape relevant content (adjust based on site structure)
        content = soup.find_all(['h2', 'h3', 'h4', 'p', 'span'])  # Example: scraping paragraphs
        page_content = "\n".join([p.get_text() for p in content])
        return page_content
    except Exception as e:
        return f"Error scraping {url}: {e}"

content = scrape_subpage_content("https://www.nhl.com/penguins/schedule/2024/fullseason")
print(content)

Tickets
Team
News
Schedule


Navigation Menu
Photos
Video
Scores
Stats
Community
Fans
Game Day
Shop
League
Help
NHL.com
  

  


Schedule Home

Promotional Schedule
Schedule Downloads


X


Instagram


Facebook


YouTube


TikTok


LinkedIn
Privacy Policy
Contact Us
NHL.com Terms of Service
NHL.com Privacy Policy
Cookie Policy
Cookie Settings
Copyright Policy
Employment

Close


In [28]:
import time


sublinks = []
fests = " "
base_link = "https://www.sportspittsburgh.com/power-of-sports-economy/professional-sports-teams/"
for i in range(1,21):
    driver = webdriver.Chrome()
    if i == 1:
        driver.get(base_link)
    else:
        driver.get(base_link + f"?page={i}")
    # Scroll the page to load all the events
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for the page to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    # Get the page source after interactions
    page_source = driver.page_source
    # Parse the page source with Beautiful Soup
    soup = BeautifulSoup(page_source, "html.parser")
    # Extract data using Beautiful Soup methods
    events = soup.find_all('div', class_='card')
    for event in events:
        details = event.find('div', class_= "card__main").find_all(['h2', 'h3', 'h4', 'p', 'span'])
        if details:
            fests += " ".join([d.text.strip() for d in details if d])
        link = event.find('a')
        if link['href'].startswith('/events/'):
            sublinks.append(link['href'])
    driver.quit()

In [30]:
print(sublinks)

[]
